In [ ]:
import itertools
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
import numpy as np
import matplotlib.pyplot as plt


In [ ]:

# Define device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Function to get the arithmetic operation
def get_arithmetic_func(name):
    return {
        '+': lambda x, y, mod: (x, y, (x + y) % mod),
    }[name]

# Vocabulary class for encoding and decoding tokens
class Vocab:
    def _init_(self, tokens):
        self.stoi = {t: idx for idx, t in enumerate(sorted(tokens))}
        self.itos = {idx: t for idx, t in enumerate(sorted(tokens))}

    def encode(self, obj):
        return torch.tensor([self.stoi[str(s)] for s in obj], dtype=torch.long)

    def decode(self, obj):
        return ' '.join([self.itos[i.item()] for i in obj])

# Dataset class for generating and handling arithmetic sequences
class ArithmeticDataset(Dataset):
    def _init_(self, operator='+', mod=97):
        self.data, tokens = self.generate_data(operator, mod)
        self.vocab = Vocab(tokens)

    def _getitem_(self, index):
        return self.vocab.encode(self.data[index])

    def _len_(self):
        return len(self.data)

    def decode(self, item):
        return self.vocab.decode(item)

    @staticmethod
    def generate_data(operator, mod):
        function = get_arithmetic_func(operator)
        result = []
        tokens = {'<sos>', '=', str(operator)}

        tuples = itertools.product(range(mod), repeat=2)

        for x, y in tuples:
            x, y, res = function(x, y, mod)
            result.append(['<sos>', x, operator, y, '=', res])
            tokens.update({str(x), str(y), str(res)})

        return result, tokens

# Custom initialization functions
def init(model, alpha):
    state_dict = model.state_dict()
    modules = ["embedding.weight", "lstm.weight_ih_l0", "lstm.weight_hh_l0", "lstm.bias_ih_l0", "lstm.bias_hh_l0", "fc.weight", "fc.bias"]
    for module in modules:
        if module in state_dict:
            state_dict[module] = state_dict[module] * alpha
    model.load_state_dict(state_dict)

def init2(model, alpha):
    with torch.no_grad():
        for name, param in model.named_parameters():
            param.data = param.data * alpha


In [ ]:

# Recurrent Neural Network model using LSTM with optional dropout
class ArithmeticModel(nn.Module):
    def _init_(self, vocab_size, embed_size, hidden_size, output_size, dropout_rate=0.0):
        super(ArithmeticModel, self)._init_()
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.lstm = nn.LSTM(embed_size, hidden_size, batch_first=True, dropout=dropout_rate)
        self.fc = nn.Linear(hidden_size, output_size)
        self._initialize_weights()

    def _initialize_weights(self):
        # Initialize weights with custom method
        init(self, alpha=5.0)

    def forward(self, x):
        x = self.embedding(x)
        lstm_out, _ = self.lstm(x)
        out = self.fc(lstm_out[:, -1, :])
        return out

# Function to evaluate model
def evaluate_model(model, dataloader, criterion):
    model.eval()
    total_correct = 0
    total_samples = 0
    total_loss = 0.0
    with torch.no_grad():
        for batch in dataloader:
            inputs = batch[:, :-1].to(device)
            targets = batch[:, -1].to(device)

            outputs = model(inputs)
            loss = criterion(outputs, targets)
            total_loss += loss.item() * targets.size(0)

            _, predicted = torch.max(outputs, 1)
            total_correct += (predicted == targets).sum().item()
            total_samples += targets.size(0)

    accuracy = total_correct / total_samples * 100
    avg_loss = total_loss / total_samples
    return accuracy, avg_loss

# Function to calculate weight norm
def calculate_weight_norm(model):
    total_norm = 0.0
    for param in model.parameters():
        param_norm = param.data.norm(2)
        total_norm += param_norm.item() ** 2
    total_norm = total_norm ** (1. / 2)
    return total_norm

# Hyperparameters and configurations
optimizer_name = 'adam'
regularizations = ['weight_decay']
sample_sizes = [0.7]
num_epochs = 5000
batch_size = 32
embed_size = 128
hidden_size = 128

results = {}


In [ ]:

# Training and evaluation
for reg in regularizations:
    for sample_size in sample_sizes:
        config_name = f'opt_{optimizer_name}reg{reg}sample{int(sample_size*100)}'
        print(f'Training with configuration: {config_name}')

        # Prepare the dataset
        dataset = ArithmeticDataset(operator='+', mod=97)
        train_size = int(sample_size * len(dataset))
        val_size = len(dataset) - train_size
        train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
        train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
        val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

        vocab_size = len(dataset.vocab.stoi)
        output_size = vocab_size

        # Initialize model with optional dropout
        dropout_rate = 0.5 if reg == 'dropout' else 0.0
        model = ArithmeticModel(vocab_size, embed_size, hidden_size, output_size, dropout_rate).to(device)

        # Select optimizer
        optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, weight_decay=0 if reg != 'weight_decay' else 0.01)

        criterion = nn.CrossEntropyLoss()

        # Training loop
        for epoch in range(1, num_epochs + 1):
            model.train()
            total_train_loss = 0.0
            total_train_samples = 0

            for batch in train_dataloader:
                inputs = batch[:, :-1].to(device)
                targets = batch[:, -1].to(device)

                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                loss.backward()
                optimizer.step()

                total_train_loss += loss.item() * targets.size(0)
                total_train_samples += targets.size(0)

            train_accuracy, train_loss = evaluate_model(model, train_dataloader, criterion)
            val_accuracy, val_loss = evaluate_model(model, val_dataloader, criterion)
            weight_norm = calculate_weight_norm(model)

            if config_name not in results:
                results[config_name] = {
                    'train_accuracy': [],
                    'val_accuracy': [],
                    'train_loss': [],
                    'val_loss': [],
                    'weight_norm': []
                }

            results[config_name]['train_accuracy'].append(train_accuracy)
            results[config_name]['val_accuracy'].append(val_accuracy)
            results[config_name]['train_loss'].append(train_loss)
            results[config_name]['val_loss'].append(val_loss)
            results[config_name]['weight_norm'].append(weight_norm)

            print(f'Epoch {epoch}/{num_epochs}, Train Accuracy: {train_accuracy:.2f}%, Val Accuracy: {val_accuracy:.2f}%, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Weight Norm: {weight_norm:.4f}')

# Plotting results in a grid
num_configs = len(regularizations) * len(sample_sizes)
fig, axes = plt.subplots(nrows=num_configs, ncols=3, figsize=(20, 5*num_configs))

# Ensure axes is always a 2D array
if num_configs == 1:
    axes = np.expand_dims(axes, axis=0)

config_index = 0
for reg in regularizations:
    for sample_size in sample_sizes:
        config_name = f'opt_{optimizer_name}reg{reg}sample{int(sample_size*100)}'
        
        # Training and validation loss
        ax1 = axes[config_index, 0]
        ax1.plot(results[config_name]['train_loss'], label=f'Train Loss')
        ax1.plot(results[config_name]['val_loss'], label=f'Val Loss')
        ax1.set_xlabel('Epoch')
        ax1.set_ylabel('Loss')
        ax1.set_title(f'Training and Validation Loss - {config_name}')
        ax1.legend()

        # Training and validation accuracy
        ax2 = axes[config_index, 1]
        ax2.plot(results[config_name]['train_accuracy'], label=f'Train Accuracy')
        ax2.plot(results[config_name]['val_accuracy'], label=f'Val Accuracy')
        ax2.set_xlabel('Epoch')
        ax2.set_ylabel('Accuracy (%)')
        ax2.set_title(f'Training and Validation Accuracy - {config_name}')
        ax2.legend()

        # Weight norm
        ax3 = axes[config_index, 2]
        ax3.plot(results[config_name]['weight_norm'], label=f'Weight Norm')
        ax3.set_xlabel('Epoch')
        ax3.set_ylabel('Weight Norm')
        ax3.set_title(f'Weight Norm - {config_name}')
        ax3.legend()

        config_index += 1

plt.tight_layout()
plt.show()
